In [ ]:

import argparse

import pytorch_lightning as pl
from pytorch_lightning.core.saving import save_hparams_to_yaml
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader

from aiscli.load import load_callbacks
from user_lightning_model import UserLightningModule
from user_datamodule import UserDataModule

TRAINER_HPARAMS = {'num_processes': 1, 'gpus': [0], 'auto_select_gpus': False, 'track_grad_norm': -1, 'check_val_every_n_epoch': 1, 'fast_dev_run': False, 'accumulate_grad_batches': 1, 'max_epochs': 10, 'min_epochs': 1, 'precision': 32, 'weights_summary': 'top', 'auto_lr_find': False, 'auto_scale_batch_size': False, 'move_metrics_to_cpu': False}

if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    parser.add_argument('--ckpt_path', type=str, default=None)
    parser.add_argument('--gpu_num', type=int, default=0)
    args = parser.parse_args()

    device = torch.device("cuda:%s" % args.gpu_num if torch.cuda.is_available() else "cpu")
    print('inference using device:%s ' % device)

    if ckpt_path is None:
        ckpt_path = get_best_ckpt_path()
    model = UserModel().to(device)
    model = model.load_from_checkpoint(checkpoint_path=ckpt_path)

    total_pred = []
    total_y = []
    for x, y in UserDataModule().test_dataloader:
        pred = model(x.to(device))
        total_pred.append(pred.detach().cpu())
        total_y.append(y)

    total_pred = torch.cat(total_pred)
    total_y = torch.cat(total_y)

    metric = torch.nn.CrossEntropyLoss()
    print(metric(total_pred, total_y).item())
